In [1]:
import sys
sys.path.append('../../')

In [2]:
import torch
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

from models.train import train, test
from models.models import GCN, GAT, GraphSAGE, FNN
from models.utils import ContagionDataset
from sklearn.metrics import matthews_corrcoef, mean_squared_error

Using backend: pytorch


In [3]:
def pretty(ld, indent=0):
    with open('result.txt', 'w', encoding='utf-8') as file:
        for d in tqdm(ld):
            file.write('{' + '\n')
            for key, value in d.items():
                file.write('\t' * (indent+1) + str(key) + ':' + str(value) + '\n')
                # file.write('\t' * (indent+1) + str(key) + '\n')
                # file.write('\t' * (indent+2) + str(value) + '\n')
            file.write('},\n')

In [4]:
do_train = False
seed = 4444

# Small dataset

In [5]:
data_dir = './data'
log_path = './logs'
save_path = './saved'

## Small network: additional stress

In [6]:
dict_small_acc_train = {}
dict_small_acc_val = {}
dict_small_acc_test = {}
dict_small_rmse_train = {}
dict_small_rmse_val = {}
dict_small_rmse_test = {}
dict_small_mcc_train = {}
dict_small_mcc_val = {}
dict_small_mcc_test = {}
sets_lengths = (0.7, 0.15, 0.15)
target = 'additional_stress'

In [7]:
dataset = ContagionDataset(
    raw_dir=data_dir,
    drop_edges=0,
    sets_lengths=sets_lengths,
    target = target,
    seed=seed,
)
x_train,x_test,y_train,y_test = train_test_split(dataset.node_features[0].to_numpy(), dataset.targets[0], test_size=0.2, random_state=seed)

### Logistic Regression

In [8]:
model_lr = LogisticRegression(multi_class='multinomial',random_state=seed, max_iter=200).fit(x_train, y_train)
print(classification_report(y_true=y_test, y_pred=model_lr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         4
           1       0.50      0.12      0.20         8
           2       0.36      0.62      0.45         8
           3       0.00      0.00      0.00         5

    accuracy                           0.40        25
   macro avg       0.38      0.44      0.36        25
weighted avg       0.38      0.40      0.34        25



In [9]:
dict_small_acc_train['logistic_regression'] = model_lr.score(x_train, y_train)
dict_small_acc_test['logistic_regression'] = model_lr.score(x_test, y_test)
print(f"Train accuracy: {dict_small_acc_train['logistic_regression']}")
print(f"Test accuracy: {dict_small_acc_test['logistic_regression']}")

dict_small_rmse_train['logistic_regression'] = mean_squared_error(y_true=y_train,y_pred=model_lr.predict(x_train), squared=False)
dict_small_rmse_test['logistic_regression'] = mean_squared_error(y_true=y_test,y_pred=model_lr.predict(x_test), squared=False)
print(f"Train rmse: {dict_small_rmse_train['logistic_regression']}")
print(f"Test rmse: {dict_small_rmse_test['logistic_regression']}")

dict_small_mcc_train['logistic_regression'] = matthews_corrcoef(y_true=y_train,y_pred=model_lr.predict(x_train))
dict_small_mcc_test['logistic_regression'] = matthews_corrcoef(y_true=y_test,y_pred=model_lr.predict(x_test))
print(f"Train mcc: {dict_small_mcc_train['logistic_regression']}")
print(f"Test mcc: {dict_small_mcc_test['logistic_regression']}")

Train accuracy: 0.47
Test accuracy: 0.4
Train rmse: 0.9848857801796105
Test rmse: 0.848528137423857
Train mcc: 0.296333277691352
Test mcc: 0.19939022926723152


### Random Forest

In [10]:
x_train_rf,x_val_rf,y_train_rf,y_val_rf = train_test_split(x_train, y_train, test_size=0.2, random_state=seed)

In [11]:
n = 20
num_nodes = x_train_rf.shape[0]
model_rf = None
val_acc = 0.0
for k in trange(1,num_nodes, (num_nodes - 1) // n):
    tmp = RandomForestClassifier(random_state=seed, n_estimators=k).fit(x_train_rf,y_train_rf)
    tmp_acc = tmp.score(x_val_rf, y_val_rf)
    if val_acc < tmp_acc:
        val_acc = tmp_acc
        model_rf = tmp

print(classification_report(y_true=y_test, y_pred=model_rf.predict(x_test)))

100%|██████████| 27/27 [00:00<00:00, 28.95it/s]

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         4
           1       1.00      0.25      0.40         8
           2       0.55      0.75      0.63         8
           3       0.50      0.40      0.44         5

    accuracy                           0.56        25
   macro avg       0.64      0.60      0.54        25
weighted avg       0.67      0.56      0.53        25



In [12]:
model_rf

RandomForestClassifier(n_estimators=43, random_state=4444)

In [13]:
dict_small_acc_train['random_forest'] = model_rf.score(x_train_rf, y_train_rf)
dict_small_acc_val['random_forest'] = model_rf.score(x_val_rf, y_val_rf)
dict_small_acc_test['random_forest'] = model_rf.score(x_test, y_test)
print(f"Train accuracy: {dict_small_acc_train['random_forest']}")
print(f"Val accuracy: {dict_small_acc_val['random_forest']}")
print(f"Test accuracy: {dict_small_acc_test['random_forest']}")

dict_small_rmse_train['random_forest'] = mean_squared_error(y_true=y_train_rf,y_pred=model_rf.predict(x_train_rf), squared=False)
dict_small_rmse_val['random_forest'] = mean_squared_error(y_true=y_val_rf,y_pred=model_rf.predict(x_val_rf), squared=False)
dict_small_rmse_test['random_forest'] = mean_squared_error(y_true=y_test,y_pred=model_rf.predict(x_test), squared=False)
print(f"Train rmse: {dict_small_rmse_train['random_forest']}")
print(f"Val rmse: {dict_small_rmse_val['random_forest']}")
print(f"Test rmse: {dict_small_rmse_test['random_forest']}")

dict_small_mcc_train['random_forest'] = matthews_corrcoef(y_true=y_train_rf,y_pred=model_rf.predict(x_train_rf))
dict_small_mcc_val['random_forest'] = matthews_corrcoef(y_true=y_val_rf,y_pred=model_rf.predict(x_val_rf))
dict_small_mcc_test['random_forest'] = matthews_corrcoef(y_true=y_test,y_pred=model_rf.predict(x_test))
print(f"Train mcc: {dict_small_mcc_train['random_forest']}")
print(f"Val mcc: {dict_small_mcc_val['random_forest']}")
print(f"Test mcc: {dict_small_mcc_test['random_forest']}")

Train accuracy: 1.0
Val accuracy: 0.4
Test accuracy: 0.56
Train rmse: 0.0
Val rmse: 1.2041594578792296
Test rmse: 0.9591663046625439
Train mcc: 1.0
Val mcc: 0.20469798657718122
Test mcc: 0.44329686849489114


### KNN Classifier

In [14]:
model_knn = KNeighborsClassifier(n_neighbors=dataset.num_classes).fit(x_train,y_train)
print(classification_report(y_true=y_test, y_pred=model_knn.predict(x_test)))

              precision    recall  f1-score   support

           0       0.40      1.00      0.57         4
           1       0.00      0.00      0.00         8
           2       0.29      0.25      0.27         8
           3       0.33      0.20      0.25         5

    accuracy                           0.28        25
   macro avg       0.25      0.36      0.27        25
weighted avg       0.22      0.28      0.23        25



In [15]:
dict_small_acc_train['knn_classifier'] = model_knn.score(x_train_rf, y_train_rf)
dict_small_acc_test['knn_classifier'] = model_knn.score(x_test, y_test)
print(f"Train accuracy: {dict_small_acc_train['knn_classifier']}")
print(f"Test accuracy: {dict_small_acc_test['knn_classifier']}")

dict_small_rmse_train['knn_classifier'] = mean_squared_error(y_true=y_train_rf,y_pred=model_knn.predict(x_train_rf), squared=False)
dict_small_rmse_test['knn_classifier'] = mean_squared_error(y_true=y_test,y_pred=model_knn.predict(x_test), squared=False)
print(f"Train rmse: {dict_small_rmse_train['knn_classifier']}")
print(f"Test rmse: {dict_small_rmse_test['knn_classifier']}")

dict_small_mcc_train['knn_classifier'] = matthews_corrcoef(y_true=y_train_rf,y_pred=model_knn.predict(x_train_rf))
dict_small_mcc_test['knn_classifier'] = matthews_corrcoef(y_true=y_test,y_pred=model_knn.predict(x_test))
print(f"Train mcc: {dict_small_mcc_train['knn_classifier']}")
print(f"Test mcc: {dict_small_mcc_test['knn_classifier']}")

Train accuracy: 0.6375
Test accuracy: 0.28
Train rmse: 0.85146931829632
Test rmse: 1.2806248474865698
Train mcc: 0.5151733184859322
Test mcc: 0.05345861288192787


### FNN

In [16]:
save_model = f'{save_path}_fnn'
log_dir = f'{log_path}_fnn'

dataset_val = ContagionDataset(
    raw_dir=data_dir,
    drop_edges=0,
    sets_lengths=sets_lengths,
    add_self_loop = False,
    target = target,
    seed=seed,
)

fnn_model = dict(
    in_features=[len(dataset_val.node_attributes)],
    h_features=[[5, 10], [10, 15], [5,5,5], [5, 10, 15], [5, 10, 15, 20], [5], [10], [15]],
    # h_features=[[5, 10], [10, 15], [5], [10], [15], [10,15]],
    out_features=[dataset_val.num_classes],
    activation=[torch.nn.ReLU()],
    norm_nodes = [None, 'bn', 'gn'],
    dropout=[0.2, 0.5, 0.0],
    # other
    lr=[1, 1e-1, 1e-2],
    label_smoothing=[0.0, 0.2, 0.4],
)
list_model = [dict(zip(fnn_model.keys(), k)) for k in itertools.product(*fnn_model.values())]

In [19]:
if do_train:
    for d in tqdm(list_model):
        d = d.copy()
        lr = d.pop('lr')
        ls = d.pop('label_smoothing')

        train(
            model=FNN(**d),
            dict_model=d,
            dataset_train=dataset_val,
            dataset_val=dataset_val,
            log_dir=log_dir,
            save_path=save_model,
            lr=lr,
            optimizer_name="adamw",
            n_epochs=100,
            scheduler_mode='max_val_acc',
            debug_mode=False,
            steps_save=10,
            use_cpu=False,
            label_smoothing=ls,
            use_edge_weight=False,
        )

#### Results

In [17]:
res_edges_fnn = test(
    dataset=dataset_val,
    save_path=save_model,
    n_runs=1,
    debug_mode=False,
    use_cpu=False,
    save=True,
    use_edge_weight=False,
)

100%|██████████| 7128/7128 [01:38<00:00, 72.17it/s] 


In [18]:
res_edges = res_edges_fnn
res_edges[0]

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'activation': ReLU(),
 'norm_nodes': 'bn',
 'dropout': 0.0,
 'train_lr': 0.1,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.4,
 'train_use_edge_weight': False,
 'train_self_loop': False,
 'train_drop_edges': 0,
 'train_loss': 1.3168471,
 'train_acc': 0.4827585816383362,
 'val_acc': 0.6111108064651489,
 'epoch': 99,
 'model_class': 'fnn',
 'path_name': '4_[10]_4_ReLU()_bn_0.0_0.1_adamw_max_val_acc_0.4_False_False_0',
 'train_mcc': 0.3207645514724477,
 'val_mcc': 0.513408188611579,
 'test_mcc': 0.5079103784286328,
 'train_rmse': 1.1695366997037857,
 'val_rmse': 0.8498365855987975,
 'test_rmse': 1.118033988749895,
 'test_acc': 0.599999725818634}

In [19]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict']['test_mcc'] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_features': 4,
 'h_features': [5, 5, 5],
 'out_features': 4,
 'activation': ReLU(),
 'norm_nodes': 'bn',
 'dropout': 0.0,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.2,
 'train_use_edge_weight': False,
 'train_self_loop': False,
 'train_drop_edges': 0,
 'train_loss': 1.2815459,
 'train_acc': 0.40229880809783936,
 'val_acc': 0.4444442093372345,
 'epoch': 20,
 'model_class': 'fnn',
 'path_name': '4_[5_5_5]_4_ReLU()_bn_0.0_0.01_adamw_max_val_acc_0.2_False_False_0_20',
 'train_mcc': 0.2288487810963607,
 'val_mcc': 0.2744106499742259,
 'test_mcc': 0.5573530100003692,
 'train_rmse': 1.4582307024641867,
 'val_rmse': 1.5456030825826172,
 'test_rmse': 1.161895003862225,
 'test_acc': 0.599999725818634}

In [28]:
pretty([all[k]['dict'] for k in sort_idx])

100%|██████████| 7128/7128 [00:00<00:00, 23510.07it/s]


### Baseline Overall

In [122]:
print(dict_small_acc_train)
print(dict_small_acc_val)
print(dict_small_acc_test)

{'logistic_regression': 0.47, 'random_forest': 1.0, 'knn_classifier': 0.6375}
{'random_forest': 0.4}
{'logistic_regression': 0.4, 'random_forest': 0.56, 'knn_classifier': 0.28}


In [124]:
print(dict_small_mcc_train)
print(dict_small_mcc_val)
print(dict_small_mcc_test)

{'logistic_regression': 0.296333277691352, 'random_forest': 1.0, 'knn_classifier': 0.5151733184859322}
{'random_forest': 0.20469798657718122}
{'logistic_regression': 0.19939022926723152, 'random_forest': 0.44329686849489114, 'knn_classifier': 0.05345861288192787}


In [123]:
print(dict_small_rmse_train)
print(dict_small_rmse_val)
print(dict_small_rmse_test)

{'logistic_regression': 0.9848857801796105, 'random_forest': 0.0, 'knn_classifier': 0.85146931829632}
{'random_forest': 1.2041594578792296}
{'logistic_regression': 0.848528137423857, 'random_forest': 0.9591663046625439, 'knn_classifier': 1.2806248474865698}


### GCN

In [9]:
save_model = f'{save_path}_gcn'
log_dir = f'{log_path}_gcn'

add_self_loop = True

dataset_val = ContagionDataset(
    raw_dir=data_dir,
    drop_edges=0,
    sets_lengths=sets_lengths,
    add_self_loop = add_self_loop,
    target = target,
    seed=seed,
)

# gcn_model = dict(
#     in_features=[len(dataset_val.node_attributes)],
#     h_features=[[5, 10], [10, 15], [5,5,5], [5, 10, 15], [5, 10, 15, 20], [5], [10], [15]],
#     # h_features=[[5, 10], [10, 15], [5], [10], [15], [10,15]],
#     out_features=[dataset_val.num_classes],
#     activation=[torch.nn.ReLU()],
#     norm_edges=['both', 'none'],
#     norm_nodes=[None, 'bn', 'gn'],
#     dropout=[0.2, 0.5, 0.0],
#     # other
#     lr=[1],
#     label_smoothing=[0.0, 0.2, 0.4],
#     use_edge_weight=[True, False],
#     drop_edges=[0,0.2,0.4],
# )
# list_model = [dict(zip(gcn_model.keys(), k)) for k in itertools.product(*gcn_model.values())]

gcn_model = dict(
    in_features=[len(dataset_val.node_attributes)],
    h_features=[[5, 10], [10, 15], [5,5,5], [5, 10, 15], [5, 10, 15, 20], [5], [10], [15]],
    # h_features=[[5, 10], [10, 15], [5], [10], [15], [10,15]],
    out_features=[dataset_val.num_classes],
    activation=[torch.nn.ReLU()],
    norm_edges=['both', 'none'],
    norm_nodes=[None, 'bn', 'gn'],
    dropout=[0.2, 0.5, 0.0],
    # other
    lr=[1],
    label_smoothing=[0.0, 0.2, 0.4],
    use_edge_weight=[True, False],
    drop_edges=[0,0.2,0.4],
)
list_model = [{i:j[k] for i,j in gcn_model.items()} for k in range(len(gcn_model['in_features']))]

In [82]:
if do_train:
    for d in tqdm(list_model):
        d = d.copy()
        lr = d.pop('lr')
        ls = d.pop('label_smoothing')
        drop_edges = d.pop('drop_edges')
        use_edge_weight = d.pop('use_edge_weight')

        # dataset_valid = ContagionDataset(
        #     raw_dir=data_dir,
        #     drop_edges=0,
        #     sets_lengths=sets_lengths,
        #     add_self_loop = add_self_loop,
        #     target = target,
        #     seed=seed,
        # )

        dataset_train = ContagionDataset(
            raw_dir=data_dir,
            drop_edges=drop_edges,
            sets_lengths=sets_lengths,
            add_self_loop = add_self_loop,
            target = target,
            seed=seed,
        )

        train(
            model=GCN(**d),
            dict_model=d,
            dataset_train=dataset_train,
            dataset_val=dataset_val,
            log_dir=log_dir,
            save_path=save_model,
            lr=lr,
            optimizer_name="adamw",
            n_epochs=100,
            scheduler_mode='max_val_acc',
            debug_mode=False,
            steps_save=10,
            use_cpu=False,
            label_smoothing=ls,
            use_edge_weight=use_edge_weight,
        )

#### Results

In [10]:
res_edges_gcn = test(
    dataset=dataset_val,
    save_path=save_model,
    n_runs=1,
    debug_mode=False,
    use_cpu=False,
    save=True,
    use_edge_weight=True,
)

100%|██████████| 28512/28512 [08:54<00:00, 53.32it/s]


In [11]:
res_edges = res_edges_gcn
res_edges[0]

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'activation': ReLU(),
 'norm_edges': 'both',
 'norm_nodes': 'bn',
 'dropout': 0.0,
 'train_lr': 1,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.2,
 'train_use_edge_weight': True,
 'train_self_loop': True,
 'train_drop_edges': 0.2,
 'train_loss': 2.3951993,
 'train_acc': 0.4597700834274292,
 'val_acc': 0.4444442093372345,
 'epoch': 99,
 'model_class': 'gcn',
 'path_name': '4_[10]_4_ReLU()_both_bn_0.0_1_adamw_max_val_acc_0.2_True_True_0.2',
 'train_mcc': 0.311739863790194,
 'val_mcc': 0.29846642969749937,
 'test_mcc': 0.6300633823448337,
 'train_rmse': 1.364576478442026,
 'val_rmse': 1.1547005383792515,
 'test_rmse': 0.8660254037844386,
 'test_acc': 0.6999996900558472}

In [12]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict']['test_mcc'] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_features': 4,
 'h_features': [5, 10, 15, 20],
 'out_features': 4,
 'activation': ReLU(),
 'norm_edges': 'none',
 'norm_nodes': 'bn',
 'dropout': 0.5,
 'train_lr': 1,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.2,
 'train_use_edge_weight': True,
 'train_self_loop': True,
 'train_drop_edges': 0.2,
 'train_loss': 10.910927,
 'train_acc': 0.4367815852165222,
 'val_acc': 0.4444442093372345,
 'epoch': 40,
 'model_class': 'gcn',
 'path_name': '4_[5_10_15_20]_4_ReLU()_none_bn_0.5_1_adamw_max_val_acc_0.2_True_True_0.2_40',
 'train_mcc': 0.26015399709440784,
 'val_mcc': 0.26083624077908835,
 'test_mcc': 0.6474232848858422,
 'train_rmse': 1.0985884360051028,
 'val_rmse': 1.0274023338281628,
 'test_rmse': 0.6708203932499369,
 'test_acc': 0.6999996900558472}

In [162]:
pretty([all[k]['dict'] for k in sort_idx])

100%|██████████| 28512/28512 [00:01<00:00, 19024.70it/s]


In [18]:
res_no_edges_gcn = test(
    dataset=dataset_val,
    save_path=save_model,
    n_runs=1,
    debug_mode=False,
    use_cpu=False,
    save=True,
    use_edge_weight=False,
)
res_no_edges = res_no_edges_gcn
res_no_edges[0]

100%|██████████| 28512/28512 [10:38<00:00, 44.68it/s]


{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'activation': ReLU(),
 'norm_edges': 'none',
 'norm_nodes': 'bn',
 'dropout': 0.2,
 'train_lr': 1,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.0,
 'train_use_edge_weight': False,
 'train_self_loop': True,
 'train_drop_edges': 0,
 'train_loss': 1.4136184,
 'train_acc': 0.5747125744819641,
 'val_acc': 0.5555552840232849,
 'epoch': 99,
 'model_class': 'gcn',
 'path_name': '4_[10]_4_ReLU()_none_bn_0.2_1_adamw_max_val_acc_0.0_False_True_0',
 'train_mcc': 0.4363938376742836,
 'val_mcc': 0.40691118905031876,
 'test_mcc': 0.6847811731681108,
 'train_rmse': 0.8969937018449045,
 'val_rmse': 1.0540925533894598,
 'test_rmse': 0.7416198487095663,
 'test_acc': 0.7499996423721313}

### GraphSAGE

In [20]:
save_model = f'{save_path}_sage'
log_dir = f'{log_path}_sage'

dataset_val = ContagionDataset(
    raw_dir=data_dir,
    drop_edges=0,
    sets_lengths=sets_lengths,
    add_self_loop = True,
    target = target,
    seed=seed,
)

sage_model = dict(
    in_features = [len(dataset.node_attributes)],
    h_features = [[5, 10], [5,5,5], [5, 10, 15], [10]], 
    out_features = [dataset.num_classes],
    aggregator_type = ['mean', 'gcn', 'pool', 'lstm'],
    norm_edges = ['right', 'none'],
    norm_nodes = [None, 'bn', 'gn'],
    activation = [torch.nn.ReLU()],
    feat_drop = [0.2, 0],
    # other
    lr=[1e-2, 1],
    label_smoothing=[0.0, 0.4],
    use_edge_weight=[True, False],
    add_self_loop=[True, False],
    drop_edges=[0,0.4],
)
list_model = [dict(zip(sage_model.keys(), k)) for k in itertools.product(*sage_model.values())]

In [23]:
if do_train:
    for d in tqdm(list_model):
        d = d.copy()
        lr = d.pop('lr')
        ls = d.pop('label_smoothing')
        add_self_loop = d.pop('add_self_loop')
        drop_edges = d.pop('drop_edges')
        use_edge_weight = d.pop('use_edge_weight')

        dataset_valid = ContagionDataset(
            raw_dir=data_dir,
            drop_edges=0,
            sets_lengths=sets_lengths,
            add_self_loop = add_self_loop,
            target = target,
            seed=seed,
        )

        dataset_train = ContagionDataset(
            raw_dir=data_dir,
            drop_edges=drop_edges,
            sets_lengths=sets_lengths,
            add_self_loop = add_self_loop,
            target = target,
            seed=seed,
        )

        train(
            model=GraphSAGE(**d),
            dict_model=d,
            dataset_train=dataset_train,
            dataset_val=dataset_valid,
            log_dir=log_dir,
            save_path=save_model,
            lr=lr,
            optimizer_name="adamw",
            n_epochs=100,
            scheduler_mode='max_val_acc',
            debug_mode=False,
            steps_save=10,
            use_cpu=False,
            label_smoothing=ls,
            use_edge_weight=use_edge_weight,
        )

#### Results

In [21]:
res_edges_sage = test(
    dataset=dataset_val,
    save_path=save_model,
    n_runs=1,
    debug_mode=False,
    use_cpu=False,
    save=True,
    use_edge_weight=True,
)

100%|██████████| 67584/67584 [32:06<00:00, 35.08it/s]  


In [22]:
res_edges = res_edges_sage
res_edges[0]

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'aggregator_type': 'gcn',
 'norm_edges': 'right',
 'norm_nodes': 'bn',
 'activation': ReLU(),
 'feat_drop': 0,
 'train_lr': 1,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.0,
 'train_use_edge_weight': True,
 'train_self_loop': True,
 'train_drop_edges': 0,
 'train_loss': 0.90199465,
 'train_acc': 0.609195351600647,
 'val_acc': 0.6111108064651489,
 'epoch': 99,
 'model_class': 'sage',
 'path_name': '4_[10]_4_gcn_right_bn_ReLU()_0_1_adamw_max_val_acc_0.0_True_True_0',
 'train_mcc': 0.48751303878829927,
 'val_mcc': 0.4874456952165277,
 'test_mcc': 0.6987776708343479,
 'train_rmse': 0.8775619308793742,
 'val_rmse': 0.6236095644623235,
 'test_rmse': 0.8366600265340756,
 'test_acc': 0.7499996423721313}

In [23]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict']['test_mcc'] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'aggregator_type': 'lstm',
 'norm_edges': 'right',
 'norm_nodes': 'bn',
 'activation': ReLU(),
 'feat_drop': 0,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.0,
 'train_use_edge_weight': True,
 'train_self_loop': True,
 'train_drop_edges': 0,
 'train_loss': 0.45962682,
 'train_acc': 0.7586206197738647,
 'val_acc': 0.6666663289070129,
 'epoch': 50,
 'model_class': 'sage',
 'path_name': '4_[10]_4_lstm_right_bn_ReLU()_0_0.01_adamw_max_val_acc_0.0_True_True_0_50',
 'train_mcc': 0.688220180417661,
 'val_mcc': 0.5560213841340322,
 'test_mcc': 0.6987776708343479,
 'train_rmse': 0.5570860145311556,
 'val_rmse': 0.5773502691896257,
 'test_rmse': 0.6324555320336759,
 'test_acc': 0.7499996423721313}

In [31]:
pretty([all[k]['dict'] for k in sort_idx])

100%|██████████| 67584/67584 [00:02<00:00, 23254.87it/s]


### GAT

In [13]:
save_model = f'{save_path}_gat'
log_dir = f'{log_path}_gat'

add_self_loop = True

dataset_val = ContagionDataset(
    raw_dir=data_dir,
    drop_edges=0,
    sets_lengths=sets_lengths,
    add_self_loop = add_self_loop,
    target = target,
    seed=seed,
)

# gat_model = dict(
#     in_features = [len(dataset.node_attributes)],
#     h_features = [[5, 10], [10, 15], [5,5,5], [5, 10, 15], [5], [10]], 
#     out_features = [dataset.num_classes],
#     num_heads = [[len(dataset.node_attributes)] * 4],
#     norm_nodes = [None, 'bn', 'gn'],
#     activation = [None, torch.nn.ReLU()],
#     negative_slope = [0.2],
#     feat_drop = [0.2],
#     attn_drop = [0.2],
#     residual = [False, True],
#     # other
#     lr=[1e-2, 1, 1e-3],
#     label_smoothing=[0.0, 0.2, 0.4],
#     use_edge_weight=[True, False],
#     drop_edges=[0,0.2,0.4],
# )
# list_model = [dict(zip(gat_model.keys(), k)) for k in itertools.product(*gat_model.values())]

gat_model = dict(
    in_features = [len(dataset.node_attributes)],
    h_features = [[5, 10], [10, 15], [5,5,5], [5, 10, 15], [5], [10]], 
    out_features = [dataset.num_classes],
    num_heads = [[len(dataset.node_attributes)] * 4],
    norm_nodes = [None, 'bn', 'gn'],
    activation = [None, torch.nn.ReLU()],
    negative_slope = [0.2],
    feat_drop = [0.2],
    attn_drop = [0.2],
    residual = [False, True],
    # other
    lr=[1e-2, 1, 1e-3],
    label_smoothing=[0.0, 0.2, 0.4],
    use_edge_weight=[True, False],
    drop_edges=[0,0.2,0.4],
)
list_model = [{i:j[k] for i,j in gat_model.items()} for k in range(len(gat_model['in_features']))]

In [ ]:
if do_train:
    for d in tqdm(list_model):
        d = d.copy()
        lr = d.pop('lr')
        ls = d.pop('label_smoothing')
        drop_edges = d.pop('drop_edges')
        use_edge_weight = d.pop('use_edge_weight')

        # dataset_valid = ContagionDataset(
        #     raw_dir=data_dir,
        #     drop_edges=0,
        #     sets_lengths=sets_lengths,
        #     add_self_loop = add_self_loop,
        #     target = target,
        #     seed=seed,
        # )

        dataset_train = ContagionDataset(
            raw_dir=data_dir,
            drop_edges=drop_edges,
            sets_lengths=sets_lengths,
            add_self_loop = add_self_loop,
            target = target,
            seed=seed,
        )

        train(
            model=GAT(**d),
            dict_model=d,
            dataset_train=dataset_train,
            dataset_val=dataset_val,
            log_dir=log_dir,
            save_path=save_model,
            lr=lr,
            optimizer_name="adamw",
            n_epochs=100,
            scheduler_mode='max_val_acc',
            debug_mode=False,
            steps_save=10,
            use_cpu=False,
            label_smoothing=ls,
            use_edge_weight=use_edge_weight,
        )

100%|██████████| 3888/3888 [6:38:33<00:00,  6.15s/it]  


#### Results

In [14]:
res_edges_gat = test(
    dataset=dataset_val,
    save_path=save_model,
    n_runs=1,
    debug_mode=False,
    use_cpu=False,
    save=True,
    use_edge_weight=True,
)

100%|██████████| 42768/42768 [16:22<00:00, 43.52it/s]


In [15]:
res_edges = res_edges_gat
res_edges[0]

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'num_heads': [4, 4, 4, 4],
 'norm_nodes': 'bn',
 'activation': ReLU(),
 'negative_slope': 0.2,
 'feat_drop': 0.2,
 'attn_drop': 0.2,
 'residual': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.2,
 'train_use_edge_weight': False,
 'train_self_loop': True,
 'train_drop_edges': 0.4,
 'train_loss': 1.7050116,
 'train_acc': 0.42528730630874634,
 'val_acc': 0.3888886868953705,
 'epoch': 20,
 'model_class': 'gat',
 'path_name': '4_[10]_4_[4_4_4_4]_bn_ReLU()_0.2_0.2_0.2_True_0.01_adamw_max_val_acc_0.2_False_True_0.4_20',
 'train_mcc': 0.23665636477024948,
 'val_mcc': 0.21092432488269025,
 'test_mcc': 0.8783984297036039,
 'train_rmse': 1.1497126077675979,
 'val_rmse': 1.0540925533894598,
 'test_rmse': 0.6324555320336759,
 'test_acc': 0.8999995589256287}

In [16]:
all = res_edges[2]
# ascending order
sort_idx = np.argsort([k['dict']['test_mcc'] for k in all])[::-1]
all[sort_idx[0]]['dict']

{'in_features': 4,
 'h_features': [10],
 'out_features': 4,
 'num_heads': [4, 4, 4, 4],
 'norm_nodes': 'bn',
 'activation': ReLU(),
 'negative_slope': 0.2,
 'feat_drop': 0.2,
 'attn_drop': 0.2,
 'residual': True,
 'train_lr': 0.01,
 'train_optimizer_name': 'adamw',
 'train_scheduler_mode': 'max_val_acc',
 'train_label_smoothing': 0.2,
 'train_use_edge_weight': False,
 'train_self_loop': True,
 'train_drop_edges': 0.4,
 'train_loss': 1.7050116,
 'train_acc': 0.42528730630874634,
 'val_acc': 0.3888886868953705,
 'epoch': 20,
 'model_class': 'gat',
 'path_name': '4_[10]_4_[4_4_4_4]_bn_ReLU()_0.2_0.2_0.2_True_0.01_adamw_max_val_acc_0.2_False_True_0.4_20',
 'train_mcc': 0.23665636477024948,
 'val_mcc': 0.21092432488269025,
 'test_mcc': 0.8783984297036039,
 'train_rmse': 1.1497126077675979,
 'val_rmse': 1.0540925533894598,
 'test_rmse': 0.6324555320336759,
 'test_acc': 0.8999995589256287}

In [17]:
pretty([all[k]['dict'] for k in sort_idx])

100%|██████████| 42768/42768 [00:01<00:00, 33203.57it/s]


### Overall

## Target: total stress